In [5]:
import wrds
import pandas as pd

# Connect to WRDS
db = wrds.Connection()

# Define the parameters for filtering permnos
start_date = '1963-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

# Query to get all permnos that are non-financial and have a share code of 10 or 11
permno_query = f"""
    SELECT DISTINCT b.permno
    FROM crsp.msenames as b
    WHERE b.siccd < '6000' AND b.siccd >= '1000'  -- Non-financial companies (SIC code < 6000)
    AND b.shrcd IN (10, 11)  -- Share code 10 or 11
    AND b.namedt <= '{end_date}'
    AND (b.nameendt >= '{start_date}' OR b.nameendt IS NULL)
"""

# Execute the query to get the list of permnos
permnos_df = db.raw_sql(permno_query)
permnos_df.to_csv("permnos.csv")
# Extract the list of permnos
permnos_list = permnos_df['permno'].tolist()

# Close the WRDS connection
db.close()

# Display the first few permnos
print(permnos_list[:5])


Loading library list...
Done
[83264, 63618, 10896, 75262, 85187]


In [9]:
import wrds
import pandas as pd

# Connect to WRDS
db = wrds.Connection()

# Define the parameters
start_date = '1963-01-01'
end_date = pd.to_datetime('today').strftime('%Y-%m-%d')

# Initialize an empty DataFrame to store the results
all_data = pd.DataFrame()

# Iterate through each permno in the list
for permno in permnos_list[:5]:
    query = f"""
        SELECT a.date, 
               a.permno, 
               a.ret as return_excluding_dividends, 
               a.prc, 
               a.vol, 
               a.shrout, 
               b.ticker, 
               a.ret*(1 + COALESCE(d.dlstcd/100, 0)) as total_return_including_dividends, 
               b.exchcd as exchange_code, 
               b.comnam as company_name, 
               c.gvkey
        FROM crsp.msf as a
        JOIN crsp.msenames as b
        ON a.permno = b.permno
        LEFT JOIN crsp.dse as d
        ON a.permno = d.permno AND a.date = d.date
        LEFT JOIN crsp.ccmxpf_linktable as ccm
        ON a.permno = ccm.lpermno
        LEFT JOIN comp.company as c
        ON ccm.gvkey = c.gvkey
        WHERE a.permno = {permno}
        AND a.date >= '{start_date}'
        AND a.date <= '{end_date}'
        AND b.namedt <= a.date
        AND a.date <= b.nameendt
        AND ccm.linkdt <= a.date
        AND (ccm.linkenddt >= a.date OR ccm.linkenddt IS NULL)
    """

    # Execute the query for the current permno
    stock_data = db.raw_sql(query)
    
    # Check if there is any price less than 5 (in absolute terms)
    #if stock_data['prc'].abs().min() < 5:
        #continue  # Skip this permno if any price is less than 5
    
    # Sort the data by date in ascending order
    stock_data = stock_data.sort_values(by=['permno', 'date'])
    
    # Append the data to the all_data DataFrame
    all_data = pd.concat([all_data, stock_data], ignore_index=True)

# Close the WRDS connection
db.close()

# Save the final data to a CSV file
all_data.to_csv("all_permnos_filtered_sorted_data.csv", index=False)

# Display the first few rows of the combined data
all_data.head()


Loading library list...
Done


,date,permno,return_excluding_dividends,prc,vol,shrout,ticker,total_return_including_dividends,exchange_code,company_name,gvkey
0,1996-03-29,83264,NaN,-32.250,178.0,12002.0,GBCOB,NaN,3,GREIF BROTHERS CORP,005338
1,1996-04-30,83264,-0.120155,-28.375,226.0,12002.0,GBCOB,-0.120155,3,GREIF BROTHERS CORP,005338
2,1996-05-31,83264,0.057269,30.000,124.0,12002.0,GBCOB,0.057269,3,GREIF BROTHERS CORP,005338
3,1996-06-28,83264,0.095667,-32.750,37.0,12002.0,GBCOB,0.095667,3,GREIF BROTHERS CORP,005338
4,1996-07-31,83264,0.041985,-34.125,490.0,12002.0,GBCOB,0.041985,3,GREIF BROTHERS CORP,005338
